# Only for Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/pyFM

/content/drive/MyDrive/pyFM


In [ ]:
!pip install potpourri3d robust_laplacian open3d trimesh tensorboard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 858 kB 6.8 MB/s 
     |████████████████████████████████| 388 kB 60.1 MB/s 
     |████████████████████████████████| 408.6 MB 28 kB/s 
     |████████████████████████████████| 652 kB 54.5 MB/s 
     |████████████████████████████████| 1.1 MB 31.5 MB/s 
     |████████████████████████████████| 3.1 MB 33.6 MB/s 
     |████████████████████████████████| 596 kB 46.3 MB/s 
     |████████████████████████████████| 8.8 MB 49.3 MB/s 
     |████████████████████████████████| 344 kB 51.4 MB/s 
     |████████████████████████████████| 423 kB 49.5 MB/s 
     |████████████████████████████████| 54 kB 3.4 MB/s 
     |████████████████████████████████| 9.7 MB 39.4 MB/s 
     |████████████████████████████████| 1.2 MB 57.8 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
     |████████████████████████████████| 561 kB 51.6 MB/s 
     |██████████████████████████████

Before this, run split_data_DA

In [14]:
import torch

In [15]:
torch.cuda.empty_cache()

# Train and evaluate dpfm model

In [16]:
import yaml
from dann.main import main
from dann.test import test_target, test_source
from project.datasets import Tosca
from project.train_dpfm import train_net
from project.eval_dpfm import eval_net
from dann.main_test import main_test

In [17]:
cfg = yaml.safe_load(open(f"dann/config/tosca_cuts.yaml", "r"))
source_domain_path = '../data/Tosca_DA/source/train'
target_domain_path = '../data/Tosca_DA/target/train'
source_test_path = '../data/Tosca_DA/source/test'
target_test_path = '../data/Tosca_DA/target/test'
torch.cuda.empty_cache()


In [18]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Clear any logs from previous runs
# !rm -rf ./runs/ 

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [19]:
%tensorboard --logdir runs 


In [ ]:
main(source_domain_path,target_domain_path,cfg)

using dataset cache path: ../data/Tosca_DA/source/train\cache_cuts_train.pt
  --> dataset not in cache, repopulating
loading mesh cat0_partial1
loading mesh cat0_partial2
loading mesh cat0_partial3
loading mesh cat0_partial5
loading mesh cat0_partial6
loading mesh cat10_partial1
loading mesh cat10_partial2
loading mesh cat10_partial4
loading mesh cat10_partial5
loading mesh cat10_partial6
loading mesh cat1_partial1
loading mesh cat1_partial2
loading mesh cat1_partial3
loading mesh cat1_partial4
loading mesh cat2_partial1
loading mesh cat2_partial2
loading mesh cat2_partial4
loading mesh cat2_partial5
loading mesh cat2_partial6
loading mesh cat3_partial1
loading mesh cat3_partial2
loading mesh cat3_partial3
loading mesh cat3_partial6
loading mesh cat4_partial1
loading mesh cat4_partial2
loading mesh cat4_partial3
loading mesh cat4_partial4
loading mesh cat4_partial5
loading mesh cat4_partial6
loading mesh cat5_partial1
loading mesh cat5_partial2
loading mesh cat5_partial5
loading mesh c

**Note**: Change model path to your respective path

### FM evaluation

In [ ]:
pred_p2p_list_tgt, pred_p2p_list_src = main_test(source_test_path, target_test_path, 'trained_models/try1_try_dann.pt', cfg, 'dann/data/dann/test_tosca.pt', n_samples=None)


In [ ]:
#p2p_fm, distances_fm = eval_net(cfg, "project/data/saved_models_cuts_3/ep_49.pth", "data/eval/test_tosca.pt", return_pred_p2p=True, return_dist=True, mode="FM")

### CFM evaluation

In [ ]:
#p2p_cfm, distances_cfm = eval_net(cfg,"project/data/saved_models_cuts_3/ep_49.pth", "data/eval/test_tosca.pt", return_pred_p2p=True, return_dist=True, mode="CFM")

# Plot results

# Notes
- Trained on 26 shapes, tested on 6 shapes (80/20 train/test split)
- Training process seems to work well, can't tell if it's overfitting due to lack of validation implementation
- Errors:
  - WKS: 5.99
  - DPFM trained on 26 shapes: 0.66
  - DPFM trained on 60% of TOSCA: 0.50
- Compared to FM with hand-crafted descriptors (WKS): 5.99 <-> 0.66 (mean normalized geodesic error)
- Improvements:
  - validation set implementation
  - tensorboard visualization
  - lr scheduler from pytorch
  - inference is slow, mostly done on CPU, transfer to GPU
- Next steps:
  - double check all dataloaders & FM to p2p conversions
  - training on whole partial-to-full dataset (TOSCA), or mix full-to-full & partial-to-full, e.g. warm start with model pre-trained on an easier problem, i.e. full-to-full
  - combine model w/ coupled FM
  - implement gradient clipping
  - apply trained descriptors to CFM


# Change log

- Remove data augmentation in validation step
- LR scheduler decaying too fast, use reduceLROnPlateau() with average validation loss
- use average of train and val loss for evaluation
- implement tensorboard logging

# Done
- trained model for 39 epochs on all shapes, good training error, val error is very high, might be a bug